# Capstone project

In [ ]:
#librerías estandarizar y crear clusters

import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.mixture import GaussianMixture
import random
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations,product,chain

import json
import pickle

pd.options.display.max_columns=None

import warnings

# Ignorar todos los warnings
warnings.filterwarnings("ignore")

# Set the seed for reproducibility
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

In [ ]:
#sitio BBDD Helena
#df=pd.read_csv('data/datos_income_TFM.csv', sep= ',')

df = pd.read_csv('data/datos_income_TFM.csv', sep=',')
df.head(3)

In [ ]:
df.columns

In [ ]:
# Eliminar variable redundante edad diagnóstico diabetes

df = df.drop(columns=['DIABAGE3', 'X.AGE80'])
df.columns

In [ ]:
# Crear variable BMI y ponerla en intervalos: Peso y altura vienen en sistema anglosajón

df['BMI_WHO']=df['WEIGHT2']/df['HEIGHT3']**2*10000
df['BMI_WHO']=df['BMI_WHO'].round().astype('int')
df['BMI_WHO']=pd.cut(df['BMI_WHO'],bins=[0,18.5,25,30,35,40,df['BMI_WHO'].max()],
                     labels=range(1,7),
                     include_lowest=True,ordered=True)
df['BMI_WHO']

In [ ]:
# one hot encoding del BMI
'''
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
onehot_encoded = encoder.fit_transform(df[['BMI_WHO']])
# Crear un DataFrame a partir de la matriz codificada
df_encoded = pd.DataFrame(onehot_encoded, columns=encoder.get_feature_names(['BMI_WHO']))
df_concat = pd.concat([df, df_encoded], axis=1)

# poner en enteros las 6 nuevas columnas de BMI
columnas = ['BMI_WHO_1', 'BMI_WHO_2','BMI_WHO_3','BMI_WHO_4','BMI_WHO_5','BMI_WHO_6']
for col in columnas:
    df_concat[col] = df_concat[col].round().astype('int')

df_concat'''

# Partición train, validation y test

In [ ]:
from sklearn.model_selection import train_test_split
target = "DIABETE4"

x = df.drop(columns=target)
y = df[target]

# Conjunto de entrenamiento del 60%
x_temp, x_train, y_temp, y_train = train_test_split(x, y, test_size=0.60, random_state=RANDOM_SEED)

# Conjuntos de validación y prueba (cada uno del 20% del total)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.50, random_state=RANDOM_SEED)

In [ ]:
x_train.shape, x_test.shape, x_val.shape

In [ ]:
### Estandarizar el train

scaler=StandardScaler()
scaler.fit(x_train)

x_train_s=pd.DataFrame(scaler.transform(x_train),
                           index=x_train.index,columns=x_train.columns)
x_train_s.head()

In [ ]:
x_train.shape

In [ ]:
## Estandarizar x_test y x_val

scaler2=StandardScaler()
scaler2.fit(x_test)

x_test_s=pd.DataFrame(scaler2.transform(x_test),
                           index=x_test.index,columns=x_test.columns)

scaler3=StandardScaler()
scaler3.fit(x_val)
x_val_s = pd.DataFrame(scaler3.transform(x_val),
                           index=x_val.index,columns=x_val.columns)

In [ ]:
#Quitar variables redundantes con feature wiz, edad diagnóstico de diabetes, edad y BMI en continuo

import featurewiz as fwiz

outputs = fwiz.featurewiz(dataname=df, target=target, corr_limit=0.85, verbose=1, sep=',', 
		header=0,  
		dask_xgboost_flag=False, nrows=None, skip_sulov=False, skip_xgboost=True)

In [ ]:
### get list of selected features ###
features, df2 = outputs

## Transponer filas y columnas en train

Para hacer clustering de variables en vez de personas, hay que transponer filas y columnas, de este modo, las variables pasan a ser las "observaciones" y las personas, características similares en función de los que luego hacer los clusters

In [ ]:
x_train_transposed = x_train_s.T
#df_standard_T
# Transponer filas y columnas
#df_standard_T = standard_data.transpose() #esta es la que ha usado Helena
#df_standard_T

In [ ]:
## Segunda tanda de estandarización, ahora por pacientes,
#esta nos la saltamos
'''
scaler2=StandardScaler() #nuevo objeto scaler
scaler2.fit(df_standard_T)

df_T_standard=pd.DataFrame(scaler2.transform(df_standard_T),
                           index=df_standard_T.index,columns=df_standard_T.columns)
df_T_standard'''

# Crear clusters sin morir en el intento

Clustering techniques to be used:
- K-means. It is a clustering algorithm based on partitioning that groups data into k distinct groups.

Indices to be used:¶
- Silhouette Coefficient. This index measures intra-cluster cohesion and inter-cluster separation. It provides a value between -1 and 1, where values closer to 1 indicate good separation between clusters and values closer to -1 indicate poor separation. It is widely used to assess the quality of clustering.

- Calinski-Harabasz Index. This index calculates the ratio between intra-cluster dispersion and inter-cluster dispersion. Higher values of the index indicate better separation between clusters.

- Davies-Bouldin Index. This index measures the average similarity between each cluster and its nearest cluster. A lower value of the index indicates better separation between clusters.

- Dunn Index. This index measures the ratio between the minimum distance between clusters and the maximum distance within each cluster. Higher values of the index indicate better separation between clusters.


In [ ]:
#Hyperparameters #este método tiende a encontrar 2 clusters, quitar esa opción
param_grid = {
    'n_clusters': [2],  # Different numbers of clusters to be assessed
    'init': ['k-means++'], #random # Different initialisation methods
    'max_iter': [100] #200 #300 # Different maximum number of iterations
}

# Evaluation indices

evaluation_indices = {
    'Silhouette Coefficient': silhouette_score,
    'Calinski-Harabasz Index': calinski_harabasz_score,
    'Davies-Bouldin Index': davies_bouldin_score
}

def dunn_index(data, labels):
    distances = pairwise_distances(data)
    intra_cluster_distances = []
    for label in set(labels):
        cluster_points = data[labels == label]
        cluster_distances = distances[labels == label][:, labels == label]
        intra_cluster_distances.append(cluster_distances.max())
    min_inter_cluster_distance = distances[labels != labels[:, None]].min()
    dunn_index = min_inter_cluster_distance / max(intra_cluster_distances)
    return dunn_index

In [ ]:
# Ajustar KMeans antes de aplicar GridSearchCV, tarda sus 30-45 min tranquilamente
'''kmeans = KMeans()  # Crea una instancia de KMeans
kmeans.fit(df_T_standard)  # Ajusta KMeans a tus datos

# Apply GridSearchCV to find the best hyperparameters.
grid_search = GridSearchCV(kmeans, param_grid, scoring=silhouette_score, cv=5)
grid_search.fit(df_T_standard)
best_params = grid_search.best_params_'''

In [ ]:
# Probar otro índice - Guillem

'''# Ajustar KMeans para pruebas de 2 a 10 clusters, sin hacer gridsearch
results =[]

for i in range(2,20):
    kmeans2 = KMeans(n_clusters=i, init= 'k-means++',
                     max_iter=100, random_state=1)  # Crea una instancia de KMeans
    kmeans2.fit(df_standard_T)
    results.append(kmeans2.inertia_)# Ajusta KMeans a tus datos

# Graficar los resultados
plt.plot(range(2, 20), results)
plt.title('Método del codo')
plt.xlabel('Número de clusters')
plt.ylabel('Inercia')
plt.show()'''

In [ ]:
'''# Computar los clusters y el

    # Calculate the evaluation indices.
scores = {'Datos_kmeans': f'Datos_{1}_kmeans', 'Numero_Clusters': num_clusters,
          'init':best_params['init'], 'max_iter': best_params['max_iter']}
# Ajustar KMeans para pruebas de 2 a 10 clusters, sin hacer gridsearch
results2 =[]

for i in range(6,9):
    kmeans3 = KMeans(n_clusters=i, init= 'k-means++',
                     max_iter=300, random_state=1)  # Crea una instancia de KMeans
    kmeans3.fit(df_T_standard) # Ajusta KMeans a tus datos
    results.append(kmeans3.inertia_)
    labels2=kmeans3.labels_


    for index_name, index_func in evaluation_indices.items():
        score = index_func(df_T_standard, labels)
        scores[index_name] = score

    # Calculate the Dunn and Hopkins indices.
    dunn = dunn_index(df_T_standard, labels)
    scores['Dunn Index'] = dunn'''

In [ ]:
# calcular scores para 2 a 9 clusters

    # Calculate the evaluation indices.
#scores = {'Datos_kmeans': f'Datos_{1}_kmeans', 'Numero_Clusters': num_clusters,
        #  'init':best_params['init'], 'max_iter': best_params['max_iter']}

resultados = []  # Crear una lista vacía para almacenar los resultados

x_train_transposed.columns = x_train_transposed.columns.astype(str)

for i in range(2,10):
    kmeans3 = KMeans(n_clusters=i, init= 'k-means++', max_iter=100, random_state=1)  # Crea una instancia de KMeans
    kmeans3.fit(x_train_transposed) # Ajusta KMeans a tus datos
    labels2 = kmeans3.labels_  # Obtiene las etiquetas del ajuste actual de KMeans

    # Añadir las etiquetas al DataFrame como una nueva columna
    x_train_transposed['Cluster_' + str(i)] = labels2

    # Imprimir la frecuencia de observaciones por cluster
    print(f"Frecuencia de observaciones para {i} clusters:")
    print(x_train_transposed['Cluster_' + str(i)].value_counts())

    # Crear un histograma de las distribuciones
    pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
    sns.countplot(x=x_train_transposed['Cluster_' + str(i)], palette= pal)
    plt.title(f"Distribución de los clusters para {i} clusters")
    plt.xlabel("Cluster")
    plt.ylabel("Frecuencia")
    plt.show()

    # Crear un nuevo diccionario para almacenar los resultados de esta iteración
    scores = {}

    for index_name, index_func in evaluation_indices.items():
        score = index_func(x_train_transposed.drop(['Cluster_' + str(i)], axis=1), labels2)  # Usa las etiquetas actuales
        scores[index_name] = score

    # Calcular los índices de Dunn y Hopkins.
    dunn = dunn_index(x_train_transposed.drop(['Cluster_' + str(i)], axis=1), labels2)  # Usa las etiquetas actuales
    scores['Dunn Index'] = dunn

    # Añadir los resultados de esta iteración a la lista de resultados
    resultados.append(scores)

In [ ]:
#uy ojo.. que sin estandarizar la segunda vez están saliendo clusters distribuidos!!!

In [ ]:
tabla_resultados = pd.DataFrame(resultados)
tabla_resultados

Visualización de los clusters

In [ ]:
from sklearn.manifold import TSNE
import umap # pip install umap-learn

In [ ]:
desired_k = 8
kmeans = KMeans(n_clusters=8, init= 'k-means++', max_iter=100, random_state=1)  # Crea una instancia de KMeans
kmeans.fit(x_train_transposed) # Ajusta KMeans a tus datos
labels = kmeans.labels_  # Obtiene las etiquetas del ajuste actual de KMeans

# Añadir las etiquetas al DataFrame como una nueva columna
x_train_transposed["cluster"] = labels

In [ ]:
tsne = TSNE(n_components=2)
x_embedded_with_tsne = tsne.fit_transform(x_train_transposed.drop(columns="cluster"))
vis_data = pd.DataFrame(x_embedded_with_tsne, index=x_train_transposed.index)
vis_data["cluster"] = x_train_transposed["cluster"]

In [ ]:
sns.scatterplot(
    x=0, y=1, hue="cluster", data=vis_data, palette=sns.color_palette("hls", 8)
)

In [ ]:
x_embedded_with_umap = umap.UMAP(n_neighbors=8, random_state=42).fit(x_train_transposed.drop(columns="cluster"))
vis_data = pd.DataFrame(x_embedded_with_umap.embedding_, index=x_train_transposed.index)
vis_data["cluster"] = x_train_transposed["cluster"]

In [ ]:
sns.scatterplot(
    x=0, y=1, hue="cluster", data=vis_data, palette=sns.color_palette("hls", 8)
)

Otro paso: automatizar la selección del mejor número de clusters'

In [ ]:
'''Otro paso: automatizar la selección del mejor número de clusters'''

In [ ]:
x_train_transposed.to_csv('Datos_TFM_columnasclusters.csv', index=False)

In [ ]:
x_train_transposed.columns

In [ ]:
# Get the best model with the selected hyperparameters.
'''best_model = KMeans(**best_params)
best_model.fit(df_T_standard)
labels = best_model.labels_
num_clusters = len(set(labels))'''

In [ ]:
'''# Calculate the evaluation indices.
scores = {'Datos_kmeans': f'Datos_{1}_kmeans', 'Numero_Clusters': num_clusters,
          'init':best_params['init'], 'max_iter': best_params['max_iter']}
for index_name, index_func in evaluation_indices.items():
    score = index_func(df_T_standard, labels)
    scores[index_name] = score

# Calculate the Dunn and Hopkins indices.
dunn = dunn_index(df_T_standard, labels)
scores['Dunn Index'] = dunn


scores'''

In [ ]:
'''nueva_fila = pd.DataFrame([scores], columns=['Datos_kmeans', 'Numero_Clusters', 'init', 'max_iter']
                          + list(evaluation_indices.keys()) + ['Dunn Index'])

# Add the results to the dataframe
k_means_df = pd.concat([df_T_standard, nueva_fila], ignore_index=True)

# Save the results to a CSV file
k_means_df.to_csv('kmeans_results.csv', index=False)'''


In [ ]:
'''#Adding the Clusters feature to the original dataframe.
df_T_standard["clusters"]= best_model.fit_predict(df_T_standard)
df_T_standard["clusters"]= df_T_standard["clusters"].astype('category')

clusters_df = df_T_standard.copy()

# Save the results to a CSV file
#clusters_df.to_csv('clusters_results.csv', index=False)'''

In [ ]:
## Crear un diccionario con las variables de nuestro cluster de elección:  clúster 8

unicos8 = x_train_transposed['Cluster_8'].unique()

diccionario8 = {}

for valor in unicos8:
    filas = x_train_transposed.index[x_train_transposed['Cluster_8'] == valor].tolist()
    diccionario8[valor] = filas

# Ver el diccionario
for clave, valor in diccionario8.items():
    print(f'Valor: {clave} - Filas: {valor}')

df_cluster8 = pd.DataFrame.from_dict(diccionario8, orient='index')
df_cluster8


In [ ]:
diccionario8

In [ ]:
diccionario8.values()

In [ ]:
dict(zip(('a','b','c'),(11,44,77)))

In [ ]:
'''crear otro diccionario que te de máximo 1/3 de la longitud del
diccionario original'''

diccionario_sizes=dict(zip(diccionario8,
            [max(1,int(len(g)/3)) for g in diccionario8.values()]))

#sobreescribimos y le decimos que coja máximo 1 vv de cada set
#diccionario_sizes=dict(zip(diccionario8,[1]*len(diccionario8)))
#diccionario_sizes

## Una estrategia: 
Coger los mejores representantes de cada cluster: que esas 6,1, 2... que coja de cada cluster, sean las que tengan la MI/corr() más alta con nuestro predictor, o quédate solo con la mitad de cada caja con corr()/MI más alta 


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Contenedor para los mejores predictores de cada cluster
best_predictors = {}

# Diccionario de clusters
diccionario8  # Tu diccionario aquí

# df de variables predictoras
x_val_s  # Tu DataFrame aquí

for cluster_name, predictors in diccionario8.items():
    # Subconjunto de df para el cluster actual
    cluster = x_val_s[predictors]

    # Calcula la MI de cada feature en el cluster con el target
    mi = mutual_info_classif(cluster, y_val)
    
    # Crea un df con los resultados
    mi_df = pd.DataFrame({
        'variable': cluster.columns,
        'mi': mi
    })

    # Ordena el df por MI en orden descendente
    mi_df = mi_df.sort_values(by='mi', ascending=False)
    
    # Selecciona el 50% superior de las variables
    top_50_percent = mi_df[:len(mi_df)//2]
    
    # Número máximo de features a seleccionar basado en el tamaño del cluster
    max_predictors = min(len(cluster.columns), 6)
    
    # Selecciona las `max_predictors` variables con mayor Información Mutua
    top_predictors = top_50_percent[:max_predictors]
    
    # Agrega las mejores variables al diccionario de mejores predictores
    best_predictors[cluster_name] = top_predictors['variable'].tolist()

# Ahora `best_predictors` es un diccionario con los nombres de las mejores variables de cada cluster

In [ ]:
best_predictors
#Esto ya hace que las combis se hagan con unas 35 vv en vez de con 78...

# Generación de combinaciones de regresores

In [ ]:
#Definir el mínimo y máximo número de features predictoras que tendrá el modelo
#n_min=len(diccionario8)
#n_max=2*len(diccionario8)
n_min=2
n_max=10
n_min, n_max

In [ ]:
def get_custom_combinations(grouped_cols_list,max_sizes_list,n_min=None,n_max=None):

    """
    Generate all possible combinations of variables from the specified groups,
    while limiting the number of variables from each group based on max_sizes_list.
    Each combination must have at least n_min variables and at most n_max variables.

    :param grouped_cols_list: A list of lists, where each sublist represents a
    group of variables.
    :param max_sizes_list: A list of integers representing the maximum number
    of variables allowed from each group.
    :param n_max: The maximum total number of variables allowed in a combination.
    :param n_min: The minimum total number of variables allowed in a combination
    :return: A list of lists, where each sublist is a valid combination of variables.
    """

    def combinations_by_group_and_max_size(group,max_size):
        """
        Generate all combinations of variables in a group, with a maximum size
        of max_size.

        :param group: A list of variables.
        :param max_size: The maximum number of variables to include in a combination.
        :return: A list of lists, where each sublist is a combination of variables.
        """
        print(f"Group: {group}, type: {type(group)}")
        print(f"Max size: {max_size}, type: {type(max_size)}")
        return [list(c) for size in range(max_size+1) for c in combinations(group,size)]

    # Generate all valid combinations for each group:
    aux=list(map(lambda tuple_:combinations_by_group_and_max_size(*tuple_),
                 zip(grouped_cols_list,max_sizes_list)))

    # Generate all possible combinations of the combinations from each group:
    out=[list(chain(*g)) for g in product(*aux)]

    # Filter combinations to ensure they have between n_min and n_max variables:
    out=[o for o in out if (n_min is None or n_min<=len(o))
         and (n_max is None or len(o)<=n_max)]


    return out


In [ ]:
# Estrategia de generar combinaciones con la selección previa en base a top valores MI

# con best_predictors

custom_combis = get_custom_combinations(list(best_predictors.values()),
                        list(diccionario_sizes.values()),n_min,n_max)


In [ ]:
#mirar numero de combinaciones

len(custom_combis)

In [ ]:
'''Función optimizada con un generador, que produce cada combinación
a medida que se solicite en lugar de todas a la vez'''

def get_custom_combinations_opt(grouped_cols_list, max_sizes_list, n_min=None, n_max=None):
    def combinations_by_group_and_max_size(group, max_size):
        for size in range(max_size+1):
            for c in combinations(group, size):
                yield list(c)

    # Generate all valid combinations for each group:
    aux=[combinations_by_group_and_max_size(group, max_size) 
        for group, max_size in zip(grouped_cols_list, max_sizes_list)]

    # Generate all possible combinations of the combinations from each group:
    for g in product(*aux):
        combined = list(chain(*g))

        # Filter combinations to ensure they have between n_min and n_max variables:
        if (n_min is None or n_min <= len(combined)) and (n_max is None or len(combined) <= n_max):
            yield combined

In [ ]:
#Definir el mínimo y máximo número de features predictoras que tendrá el modelo
n_min=len(diccionario8)
n_max=2*len(diccionario8)
n_min, n_max

In [ ]:
custom_combis_opt = get_custom_combinations_opt(list(diccionario8.values()),
                        list(diccionario_sizes.values()),n_min,n_max)

custom_combis_opt

In [ ]:
['X.STATE', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'MEDCOST', 'CVDSTRK3', 'ASTHMA3', 'ASTHNOW', 'ADDEPEV3', 'DEAF', 'BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'FALL12MN', 'HIVRISK5', 'CIMEMLOS', 'MARITAL.5', 'RENTHOM1.3', 'EMPLOY1.3', 'EMPLOY1.8', 'X.IMPRACE.5']
['SEXVAR', 'GENHLTH', 'HLTHPLN1', 'PERSDOC2', 'CHECKUP1', 'EXERANY2', 'SLEPTIM1', 'LASTDEN4', 'EDUCA', 'FLUSHOT7', 'SEATBELT', 'EMPLOY1.1', 'EMPLOY1.6']
['CHCSCNCR', 'CHCOCNCR', 'HAVARTH4', 'CHCKDNY2', 'X.AGEG5YR', 'X.AGE80', 'MARITAL.3', 'EMPLOY1.7', 'VO_CORAZON']
['CHCCOPD2', 'RMVTETH4', 'SMOKE100', 'SMOKDAY2', 'ECIGNOW', 'LCSLAST', 'LCSNUMCG', 'MARITAL.2', 'MARITAL.4', 'MARITAL.6', 'RENTHOM1.2', 'EMPLOY1.4', 'X.IMPRACE.4', 'X.IMPRACE.6']
['DIABAGE3', 'WEIGHT2', 'X.IMPRACE.2', 'BMI', 'BMI_WHO']
['CHILDREN', 'X.INCOMG', 'MARITAL.1', 'RENTHOM1.1', 'EMPLOY1.2', 'EMPLOY1.5', 'X.IMPRACE.1', 'X.IMPRACE.3']
['HEIGHT3', 'USENOW3', 'ALCDAY5', 'DRNK3GE5', 'MAXDRNKS', 'LCSFIRST']

In [ ]:
all_combis = [
    ['X.STATE', 'SEXVAR','CHCSCNCR','X.AGEG5YR'],
    ['X.STATE', 'SEXVAR','CHCSCNCR','X.AGEG5YR','CHILDREN'],
    ['X.STATE', 'SEXVAR','CHCSCNCR','X.AGEG5YR','HEIGHT3'],
]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from tqdm import tqdm

In [ ]:
## Estrategia de V3

#RF pequeño, para evitar overfitting

hyperparameters = {'n_estimators':10,  # Número reducido de árboles
    'max_depth':5,  # Profundidad máxima limitada
    'min_samples_split':20,  # Mayor número mínimo de muestras para dividir un nodo
    'min_samples_leaf':10,
    'max_features': 'auto',
    'bootstrap': True,
    'oob_score':True}

                  
results = []

for combi in tqdm(custom_combis): #no falta la cross-validation?
    model = RandomForestClassifier(**hyperparameters)
    model.fit(x_train_s[combi], y_train)
    y_pred_train = model.predict(x_train_s[combi])
    y_pred_val = model.predict(x_val_s[combi])
    metadata = {
        "Combination":combi,
        "Hyperparameters":hyperparameters,
        "AccuracyTrain":accuracy_score(y_train, y_pred_train),
        "AccuracyTest":accuracy_score(y_val, y_pred_val),
        "PrecisionTrain":precision_score(y_train, y_pred_train),
        "PrecisionTest":precision_score(y_val, y_pred_val),
        "RecallTrain":recall_score(y_train, y_pred_train),
        "RecallTest":recall_score(y_val, y_pred_val),
        "AUCTrain":roc_auc_score(y_train, y_pred_train),
        "AUCTest":roc_auc_score(y_val, y_pred_val),
    }
    results.append(metadata)

results_table = pd.DataFrame.from_records(results)

#va a unas 30-40 combinaciones analizadas por minuto

In [ ]:
results_table.head(10)

In [ ]:
# Obtener la combinación con el mejor valor en la métrica que nos interese
num_fila=results_table['AUCTest'].argmax()
results_table['Combination'].iloc[num_fila]

In [ ]:
import sqlite3
import json

# Optimización computacional de la generación de combinaciones
Aplicar el generador en el bucle for y ir guardando los resultados en una tabla de sqllite, de tal manera que si interrumpes el bucle pq aun no ha acabado puedes consultar los resultados que llevabas de la tabla de sqllite

In [ ]:
# Create DB with results
connection = sqlite3.connect("data/results.db")
cursor = connection.cursor()

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS results(
    Combination TEXT,
    Hyperparameters TEXT,
    AccuracyTrain REAL,
    AccuracyTest REAL,
    PrecisionTrain REAL,
    PrecisionTest REAL,
    RecallTrain REAL,
    RecallTest REAL,
    AUCTrain REAL,
    AUCTest REAL)
    """
)

In [ ]:
# Ejemplo para reiniciar la tabla si no se quieren usar los resultados previos
cursor.execute(
    "DELETE FROM results"
)

In [ ]:
# Example select
cursor.execute(
    "SELECT * FROM results"
).fetchall()

In [ ]:
#RF pequeño, para evitar overfitting

hyperparameters = {}
'''n_estimators':10,  # Número reducido de árboles
    'max_depth':5,  # Profundidad máxima limitada
    'min_samples_split':20,  # Mayor número mínimo de muestras para dividir un nodo
    'min_samples_leaf':10,
    'max_features': 'auto',
    'bootstrap': True,
    'oob_score':True
                  '''
results = []

for combi in tqdm(custom_combis_opt): #no falta la cross-validation?
    model = RandomForestClassifier(**hyperparameters)
    model.fit(x_train_s[combi], y_train)
    y_pred_train = model.predict(x_train_s[combi])
    y_pred_val = model.predict(x_val_s[combi])
    metadata = {
        "Combination":combi,
        "Hyperparameters":hyperparameters,
        "AccuracyTrain":accuracy_score(y_train, y_pred_train),
        "AccuracyTest":accuracy_score(y_val, y_pred_val),
        "PrecisionTrain":precision_score(y_train, y_pred_train),
        "PrecisionTest":precision_score(y_test, y_pred_val),
        "RecallTrain":recall_score(y_train, y_pred_train),
        "RecallTest":recall_score(y_test, y_pred_val),
        "AUCTrain":roc_auc_score(y_train, y_pred_train),
        "AUCTest":roc_auc_score(y_test, y_pred_val),
    }
    cursor.execute(
        f"""
        INSERT INTO results VALUES
        (
            '{json.dumps(combi)}',
            '{json.dumps(hyperparameters)}',
            {metadata["AccuracyTrain"]},
            {metadata["AccuracyTest"]},
            {metadata["PrecisionTrain"]},
            {metadata["PrecisionTest"]},
            {metadata["RecallTrain"]},
            {metadata["RecallTest"]},
            {metadata["AUCTrain"]},
            {metadata["AUCTest"]}
        )
        """
    )
    results.append(metadata)

results_table = pd.DataFrame.from_records(results)


In [ ]:
results_table_sql = pd.read_sql("SELECT * FROM results", connection)
results_table_sql["Combination"] = results_table_sql["Combination"].apply(json.loads)
results_table_sql["Hyperparameters"] = results_table_sql["Hyperparameters"].apply(json.loads)
results_table_sql

In [ ]:
results_table

In [ ]:
# Obtener la combinación con el mejor valor en la métrica que nos interese
num_fila=results_table_sql['AUCTest'].argmax()
results_table_sql['Combination'].iloc[num_fila]

In [ ]:
np.unique(y_pred_train)

Quizá seria interesante reducir el tamaño del dataset a nivel de filas para agilizar el entrenamiento de estos pequeños modelos.

In [ ]:
# Obtener la combinación con el mejor valor en la métrica que nos interese
num_fila=results_table['AUCTest'].argmax()
results_table['Combination'].iloc[num_fila]

In [ ]:
## mirar otro número de clusters: 5

unicos5 = df_standard_T['Cluster_5'].unique()

diccionario5 = {}

for valor in unicos5:
    filas = df_standard_T.index[df_standard_T['Cluster_5'] == valor].tolist()
    diccionario5[valor] = filas

# Ver el diccionario
for clave, valor in diccionario5.items():
    print(f'Valor: {clave} - Filas: {valor}')

df_cluster5 = pd.DataFrame.from_dict(diccionario5, orient='index')
df_cluster5
#este parece que tiene menos sentido